In [ ]:
# Imports 
import pandas as pd
import itertools 
import networkx as nx
import matplotlib.pyplot as plt
import pytrends
import musicbrainzngs as m
import numpy as np

from pytrends.request import TrendReq
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
artist_genres_df = pd.read_json("../data/artist_genres.json")

In [ ]:
artist_genres_df.head()

In [ ]:
import ciso8601
import time
import re
def separate_artists(df):
    split_string = ' and |, | AND | & | featuring | FEATURING | x | X | feat | ft |,| with | feauring | feat. | / '
    ret_list = []
    res_check = set()
    for index, colab in df.iterrows():
        date = ciso8601.parse_datetime(colab['date'])
        res = [time.mktime(date.timetuple())]
        artists = re.split(split_string, colab['artist'], flags=re.IGNORECASE)
        if tuple(artists) not in res_check:
            res_check.add(tuple(artists))
            res.append(artists)
            ret_list.append(res)
    return ret_list

In [ ]:
collabs = [y for [x,y] in separate_artists(df) if len(y)>1]

In [ ]:
unique_artists = list(np.unique([artist for l in collabs for artist in l]))

In [ ]:
artists_by_id = dict({a:b for a,b in zip(unique_artists,list(range(0,len(unique_artists))))})
artists_by_id = {key:[value,0.0] for (key,value) in artists_by_id.items()}

In [ ]:
for l in collabs:
    for artist in l:
        artists_by_id[artist][1] +=1

In [ ]:
top100=sorted(artists_by_id.items(), key=lambda x: x[1][1], reverse=True)[:100]

In [ ]:
top100=dict(top100)

In [ ]:
most_collab_artists = [a for (a,b) in artists_by_id.items()]

In [ ]:
most_collab_artists

In [ ]:
list(itertools.combinations(collabs[6], 2))[0][0]

In [ ]:
artists_by_id['J. Cole']

In [ ]:
included_artists = most_collab_artists.copy()
included_artists = set(included_artists)

for art in most_collab_artists:
    for collab in collabs: 
        if art in collab:
            included_artists = included_artists.union(set(collab))
included_artists = list(included_artists)

In [ ]:
artists_by_id = dict({a:b for a,b in zip(included_artists,list(range(0,len(included_artists))))})
artists_by_id 

In [ ]:
f = open("nodes.txt", "w")

f.write("var nodes = new vis.DataSet([\n")

for artist, i in top100.items():
    f.write("\t{id: "+str(i[0])+", label: '"+artist+"'},\n") 
        
f.write("]);")
f.close()

In [ ]:
'Lil Wayne' in ['Lil Wayne', 'asdsd'] and 'Li' in ['Lil Wayne', 'asdsd']

In [ ]:
 for pair in list(itertools.combinations(list(top100.keys()), 2)):
        print(pair)

In [ ]:
artists_by_id['Lil Wayne'][0]

In [ ]:
f = open("edges.txt", "w")


f.write("var edges = new vis.DataSet([\n")

for pair in list(itertools.combinations(list(top100.keys()), 2)):
    for collab in collabs:
        if pair[0] in collab and pair[1] in collab:
                f.write("\t{from: "+str(artists_by_id[pair[0]][0])+', to: '+str(artists_by_id[pair[1]][0])+"},\n")
                break
        
f.write("]);")
f.close()

In [ ]:
collabs[6]

In [ ]:
f = open("hey.txt", "w")

f.write("series: [")

for genre in aggregate_genres.keys():
    f.write("\t{\n")
    f.write("\tname : "+"'"+genre+"'"+",\n")
    f.write("\tdata: [")
    
    for subgenre in aggregate_genres[genre] : 
        f.write("{\n")
        f.write("\t\tname: "+"'"+subgenre+"',\n")
        f.write("\t\tvalue: "+str(subgenres_popularity[subgenre])+"\n")
        f.write("\t},")
    
    f.write('\t]\n')   #  closing data 
    f.write("\t},\n")

f.write("]")

f.close()

In [ ]:
# Source : https://github.com/kevinschaich/billboard 
aggregate_genres = {
    "rock": ["symphonic rock", "jazz-rock", "heartland rock", "rap rock", "garage rock", 
           "folk-rock", "roots rock", "adult alternative pop rock", "rock roll", "punk rock", 
           "arena rock", "pop-rock", "glam rock", "southern rock", "indie rock", "funk rock", 
           "country rock", "piano rock", "art rock", "rockabilly", "acoustic rock", "progressive rock", 
           "folk rock", "psychedelic rock", "rock & roll", "blues rock", "alternative rock", 
           "rock and roll", "soft rock", "rock and indie", "hard rock", "pop/rock", "pop rock", 
           "rock", "classic pop and rock", "psychedelic", "british psychedelia", "punk", "metal", "heavy metal", 
             "brutal death metal"],
"alternative/indie": ["adult alternative pop rock", "alternative rock", "alternative metal", 
                       "alternative", "lo-fi indie", "indie", "indie folk", "indietronica", 
                       "indie pop", "indie rock", "rock and indie"],
"electronic/dance": ["dance and electronica", "electro house", "electronic", "electropop", 
                      "progressive house", "hip house", "house", "eurodance", "dancehall", "dance", "trap", "funk carioca"],
"soul": ["psychedelic soul", "deep soul", "neo-soul", "neo soul", "southern soul", 
          "smooth soul", "blue-eyed soul", "soul and reggae", "soul", "funk"],
"classical/soundtrack": ["classical", "orchestral", "film soundtrack", "composer"],
"pop": ["country-pop", "latin pop", "classical pop", "pop-metal", "orchestral pop", 
         "instrumental pop", "indie pop", "sophisti-pop", "pop punk", "pop reggae", "noah cyrus"
         "britpop", "traditional pop", "power pop", "sunshine pop", "baroque pop", "alternative r&b",
         "synthpop", "art pop", "teen pop", "psychedelic pop", "folk pop", "country pop", 
         "pop rap", "pop soul", "pop and chart", "dance-pop", "pop", "top 40", "american idol", 'breakeven', 'm2m'],
"hip-hop/rnb": ["conscious hip hop", "east coast hip hop", "hardcore hip hop", 
                 "west coast hip hop", "hiphop", "southern hip hop", "hip-hop", 
                 "hip hop", "hip hop rnb and dance hall", "contemporary r b", 
                 "gangsta rap", "rapper", "rap", "rhythm and blues", "contemporary rnb", 
                 "contemporary r&b", "rnb", "rhythm & blues","r&b", "blues", "mumble rapper", 'michael jackson'],
"disco": ["disco"],
"swing":  ["swing"],
"folk": ["contemporary folk", "folk", "bachata", "latin", "romanian"],
"country": ["country rock", "country-pop", "country pop", "contemporary country", "country"],
"jazz": ["vocal jazz", "jazz", "jazz-rock"],
"religious": ["christian", "christmas music", "gospel"],
"blues": ["delta blues", "rock blues", "urban blues", "electric blues",
           "acoustic blues", "soul blues", "country blues", "jump blues", 
           "classic rock", "blues rock", "jazz and blues", "piano blues", 
           "british blues", "british rhythm & blues", "rhythm and blues", "blues", "blues rock", "rhythm & blues"],
"reggae": ["reggae fusion", "roots reggae", "reggaeton", "pop reggae", "reggae", "soul and reggae"]}

In [ ]:
all_subgenres=set([i for i in list(artist_genres_df.genres.values) for i in i])

In [ ]:
[i for i in artist_genres_df.loc[artist_genres_df['artist'] == 'Lil Wayne'].genres for i in i]

In [ ]:
df = pd.read_csv("../data/billboard.csv", index_col = 0)
df = df[df.date >= "2000-01-01"]
df

In [ ]:
def determine_genre(subgenres) :
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                return genre
    return np.NaN

```
def from_artist_to_subgenres(artist):
    subgenres = artist_genres_df.loc[artist_genres_df['artist'] == artist].genres 
    if subgenres.empty:
        return np.NaN
    else :
        return [i for i in subgenres for i in i]
```

In [ ]:
df['subgenres'] = df['artist'].map(from_artist_to_subgenres)

In [ ]:
df

<h4>Weight formula applied to determine the subgenre popularity </h4>



Popularity of each subgenre is initialized at 0.  
$s$ is a linear score given accordingly to the rank of the subgenre occurrence.


Then, for each occurrence of the genre in the charts we have :  
$ \widehat{popularity_{subgenre}} = popularity_{subgenre} +s$
  
 where $s = 1 / rank $

In [ ]:
subgenres_popularity = dict({e:0 for e in [i for i in list(aggregate_genres.values()) for i in i]})

In [ ]:
for index, row in df.loc[pd.notna(df['subgenres'])].iterrows():
    for subg in row['subgenres']:
        try : 
            subgenres_popularity[subg] += 1/row['rank']
        except KeyError:
            continue
        
    
    # print(index, row['rank'], row['subgenres'])

In [ ]:
# subgenres_popularity = {key:value for (key,value) in subgenres_popularity.items() if value>0}

In [ ]:
f = open("hey.txt", "w")

f.write("series: [")

for genre in aggregate_genres.keys():
    f.write("\t{\n")
    f.write("\tname : "+"'"+genre+"'"+",\n")
    f.write("\tdata: [")
    
    for subgenre in aggregate_genres[genre] : 
        f.write("{\n")
        f.write("\t\tname: "+"'"+subgenre+"',\n")
        f.write("\t\tvalue: "+str(subgenres_popularity[subgenre])+"\n")
        f.write("\t},")
    
    f.write('\t]\n')   #  closing data 
    f.write("\t},\n")

f.write("]")

f.close()

In [ ]:
separate_artists(artist_genres_df)